In [1]:
# This is the part 2 that will go from steps 10 until 16.
""" 
1- Read data into a dataframe (mma)
2- Separate data into train/test (80/20) (mma_train , mma_test)
3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)
4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)
5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data
6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq
7- Add DepartmentID|AisleID to solution table by joining with main table
8- Create SubstituteIDs for each ProductID
    >> For each AisleID|DepartmentID group:
        1)Select all ProductIDs | ProductNames under Aisle|Department
        2)Do a similarity analysis, and try to cluster similar product names
        3)On each cluster, compare frequency of product in orders, and select the most frequent product as SubstituteID
            Left Join with frequency table, and compare
        4)For outliers (products that are not similar to any other product), copy ProductID as SubstituteID
        5)Append ProductIDs|SubstituteIDs to Substitute table
9- Add SubstituteIDs to solution table by joining with Substitute table
10- Prepare objective function as a method
11- Prepare constraints as a method
12- Configure optimization with Objective, Constraints and Solution Table
13- Run optimization
14- Export solution table as csv
15- Test solution table with test data
16- Calculate score against metrics
"""

' \n1- Read data into a dataframe (mma)\n2- Separate data into train/test (80/20) (mma_train , mma_test)\n3- Calculate frequency in train data | test data of each product over all orders (mma_train_freq | mma_test_freq)\n4- Create a solution table with columns (ProductID|SubstituteID|PFrequency|DepartmentID|AisleID|Selected?) (solution_df)\n5- Populate the solution table ProductID column with a {SELECT DISTINCT ProductID} over original data\n6- Add information of product frequency on orders per ProductID to solution table (PFrequency), by joining with mma_train_freq\n7- Add DepartmentID|AisleID to solution table by joining with main table\n8- Create SubstituteIDs for each ProductID\n    >> For each AisleID|DepartmentID group:\n        1)Select all ProductIDs | ProductNames under Aisle|Department\n        2)Do a similarity analysis, and try to cluster similar product names\n        3)On each cluster, compare frequency of product in orders, and select the most frequent product as Substit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import scipy as sp
import sklearn as sk
import seaborn as sns
import re
import string
import math
import random
import itertools
import time
import pulp
import os
import warnings
import string
warnings.filterwarnings('ignore')


In [3]:
### Read data from Part 1
mma_test = pd.read_csv('data/processed/to_optimizer/mma_test.csv')
mma_train = pd.read_csv('data/processed/to_optimizer/mma_train.csv')
solution_df = pd.read_csv('data/processed/to_optimizer/solution_df.csv')

### 10- Prepare objective function as a method

In [4]:
def metric1(mma_train, solution_df):
    """ 
    Metric 1 : Number of orders that use in instabasket aisle items
    """
    # Select all orders that use in-aisle items
    
    
    return m1

def metric2(mma_train, solution_df):
    """ 
    Metric 1 : Number of orders that use in instabasket aisle items
    """
    # Select all orders that use in-aisle items
    
    
    return m2

### 11- Prepare constraints as methods

In [5]:
def freezer_constraint(df):
    """
    The freezer constraint limits the amount of freezer products on the resulting list to a maximum of 100
    """

    if df.loc[df['department'] == 'frozen']['Selected?'].sum() <= 100:
        return True
    else:
        return False
    
def refrigerator_constraint(df):
    """
    The refrigerator constraint limits the amount of refrigerated products on the resulting list to a maximum of 100
    """
    if df.loc[(df['department'] == 'meat seafood' ) | (df['department'] == 'dairy eggs')]['Selected?'].sum() <= 100:
        return True
    else:
        return False

def selections_constraint(df):
    """
    The selections constraint limits the amount of selections resulting list to a maximum of 1000
    """
    if df['Selected'].sum() <= 1000:
        return True
    else:
        return False

### 12- Configure optimization with Objective, Constraints and Solution Table

### 13- Run optimization

### 14

### 15- Test solution table with test data

In [6]:
### METHODS FROM MMA_MART_TEMPLATE for testing ###

# Methods for testing the solution
def tot_order(mma_test):
    # calculate total unique orders in the test data
    tot_order = mma_test['order_id'].nunique()
    print(f"Total number of orders: {tot_order}")
    return tot_order

def avg_item(mma_test, tot_order):
    # calculate the average number of items in each order
    avg_item = len(mma_test) / tot_order
    print(f"Average number of items in each order: {avg_item}")
    return avg_item

def test_my_pick(mma_test, solution_df):
    # join the test data with my pick on 'product_id'
    test_my_pick = mma_test.merge(solution_df.loc[solution_df['Selected?']==1], left_on='product_id', right_on='ProductID', how='left')

    return test_my_pick

def metrics_tmp(test_my_pick):
    # group by 'order_id' and count the non-null values of 'product_name_y' column
    metrics_tmp = test_my_pick[['order_id', 'product_name_y']].groupby('order_id').count().reset_index().rename(columns={'product_name_y': "num_prod_matching"})
    
    return metrics_tmp

def metric1(metrics_tmp, tot_order):
    # count orders with at least a match
    metric_1 = len(metrics_tmp[metrics_tmp['num_prod_matching']>0])
    print(f"Out of {tot_order} orders, {metric_1} of them utilize the in-aisle items. That's about {metric_1/tot_order*100:.2f}%.")
    return metric1

def metric2(metrics_tmp, avg_item):
    # calculate metric 2
    metric_2 = metrics_tmp['num_prod_matching'].sum() / len(metrics_tmp)
    print(f"On average there are {avg_item:.2f} items in each order, and {metric_2:.2f} of them utilize in-aisle items. That's about {metric_2/avg_item*100:.2f}%")
    return metric_2

def test(mma_test, solution_df):
    tot_ord = tot_order(mma_test)
    avg_it = avg_item(mma_test, tot_ord)
    test_my_pic = test_my_pick(mma_test, solution_df)
    metrics_tp = metrics_tmp(test_my_pic)

    metric_1 = metric1(metrics_tp, tot_ord)
    metric_2 = metric2(metrics_tp, avg_it)

    return metric_1, metric_2

In [7]:
metric_1, metric_2 = test(mma_test, solution_df)

Total number of orders: 19584
Average number of items in each order: 10.136540032679738
Out of 19584 orders, 0 of them utilize the in-aisle items. That's about 0.00%.
On average there are 10.14 items in each order, and 0.00 of them utilize in-aisle items. That's about 0.00%
